# 레시피 20 : Method Chaining 소개

In [8]:
# 간단한 메서드 체이닝 예시
import polars as pl
import seaborn as sns

# iris 데이터셋 로드
df = pl.from_pandas(sns.load_dataset('iris'))

# select와 sort를 체이닝으로 연결
result = (df
    .select(['sepal_length', 'sepal_width', 'species'])
    .sort('sepal_length', descending=True)
)

print(result.head(3))

shape: (3, 3)
┌──────────────┬─────────────┬───────────┐
│ sepal_length ┆ sepal_width ┆ species   │
│ ---          ┆ ---         ┆ ---       │
│ f64          ┆ f64         ┆ str       │
╞══════════════╪═════════════╪═══════════╡
│ 7.9          ┆ 3.8         ┆ virginica │
│ 7.7          ┆ 3.8         ┆ virginica │
│ 7.7          ┆ 2.6         ┆ virginica │
└──────────────┴─────────────┴───────────┘


# 레시피 21 : 데이터 파이프라인

In [4]:
# 1. 기본적인 데이터 전처리 파이프라인
df_processed = (df
    .select(['sepal_length', 'sepal_width', 'petal_length', 'species'])  # 필요한 컬럼만 선택
    .filter(pl.col('sepal_length') > 5.0)  # sepal_length가 5.0보다 큰 행만 필터링
    .with_columns([
        pl.col('sepal_length').mean().alias('avg_sepal_length'),  # 평균 sepal_length 컬럼 추가
        pl.col('petal_length').mean().alias('avg_petal_length')   # 평균 petal_length 컬럼 추가
    ])
)

print("기본 전처리된 데이터:")
print(df_processed.head(3))

기본 전처리된 데이터:
shape: (3, 6)
┌──────────────┬─────────────┬──────────────┬─────────┬──────────────────┬──────────────────┐
│ sepal_length ┆ sepal_width ┆ petal_length ┆ species ┆ avg_sepal_length ┆ avg_petal_length │
│ ---          ┆ ---         ┆ ---          ┆ ---     ┆ ---              ┆ ---              │
│ f64          ┆ f64         ┆ f64          ┆ str     ┆ f64              ┆ f64              │
╞══════════════╪═════════════╪══════════════╪═════════╪══════════════════╪══════════════════╡
│ 5.1          ┆ 3.5         ┆ 1.4          ┆ setosa  ┆ 6.129661         ┆ 4.315254         │
│ 5.4          ┆ 3.9         ┆ 1.7          ┆ setosa  ┆ 6.129661         ┆ 4.315254         │
│ 5.4          ┆ 3.7         ┆ 1.5          ┆ setosa  ┆ 6.129661         ┆ 4.315254         │
└──────────────┴─────────────┴──────────────┴─────────┴──────────────────┴──────────────────┘


# 레시피 22 : 집계 파이프라인

In [9]:
# 2. 집계 파이프라인
df_aggregated = (df
    .group_by('species')
    .agg([
        pl.col('sepal_length').mean().alias('avg_sepal_length'),
        pl.col('petal_length').mean().alias('avg_petal_length'),
        pl.col('sepal_width').std().alias('std_sepal_width')
    ])
    .sort('avg_sepal_length', descending=True)
)

print("\n종별 집계 데이터:")
print(df_aggregated)


종별 집계 데이터:
shape: (3, 4)
┌────────────┬──────────────────┬──────────────────┬─────────────────┐
│ species    ┆ avg_sepal_length ┆ avg_petal_length ┆ std_sepal_width │
│ ---        ┆ ---              ┆ ---              ┆ ---             │
│ str        ┆ f64              ┆ f64              ┆ f64             │
╞════════════╪══════════════════╪══════════════════╪═════════════════╡
│ virginica  ┆ 6.588            ┆ 5.552            ┆ 0.322497        │
│ versicolor ┆ 5.936            ┆ 4.26             ┆ 0.313798        │
│ setosa     ┆ 5.006            ┆ 1.462            ┆ 0.379064        │
└────────────┴──────────────────┴──────────────────┴─────────────────┘


In [ ]:
# 3. 조건부 파이프라인
df_conditional = (df
    .with_columns([
        pl.when(pl.col('sepal_length') > 6.0)
        .then('large')
        .otherwise('small')
        .alias('size_category')
    ])
    .groupby(['species', 'size_category'])
    .count()
    .sort(['species', 'size_category'])
)

print("\n조건부 분류 결과:")
print(df_conditional)

In [1]:
import polars as pl
import seaborn as sns 

# seaborn에서 iris 데이터셋을 불러와서 polars DataFrame으로 변환
df = pl.from_pandas(sns.load_dataset('iris'))
df.head(1)


sepal_length,sepal_width,petal_length,petal_width,species
f64,f64,f64,f64,str
5.1,3.5,1.4,0.2,"""setosa"""


In [3]:
%%time
df = pl.from_pandas(sns.load_dataset('iris'))

# select, filter, sort를 독립적으로 사용
df = df.select(['species', 'petal_length', 'petal_width'])

# 그 다음 petal_length가 4.0보다 큰 행만 필터링
df = df.filter(pl.col('petal_length') > 4.0)

# 마지막으로 petal_width를 기준으로 내림차순 정렬
df = df.sort('petal_width', descending=True)

print("필터링 및 정렬된 iris 데이터:")
print(df.head(5))

필터링 및 정렬된 iris 데이터:
shape: (5, 3)
┌───────────┬──────────────┬─────────────┐
│ species   ┆ petal_length ┆ petal_width │
│ ---       ┆ ---          ┆ ---         │
│ str       ┆ f64          ┆ f64         │
╞═══════════╪══════════════╪═════════════╡
│ virginica ┆ 6.0          ┆ 2.5         │
│ virginica ┆ 6.1          ┆ 2.5         │
│ virginica ┆ 5.7          ┆ 2.5         │
│ virginica ┆ 5.1          ┆ 2.4         │
│ virginica ┆ 5.6          ┆ 2.4         │
└───────────┴──────────────┴─────────────┘
CPU times: total: 0 ns
Wall time: 3.44 ms


In [4]:
%%time
df = pl.from_pandas(sns.load_dataset('iris'))

# select, filter, sort를 메소드 체이닝으로 사용
df = (df
    .select(['species', 'petal_length', 'petal_width'])
    .filter(pl.col('petal_length') > 4.0)
    .sort('petal_width', descending=True)
)

print("필터링 및 정렬된 iris 데이터:")
print(df.head(5))

필터링 및 정렬된 iris 데이터:
shape: (5, 3)
┌───────────┬──────────────┬─────────────┐
│ species   ┆ petal_length ┆ petal_width │
│ ---       ┆ ---          ┆ ---         │
│ str       ┆ f64          ┆ f64         │
╞═══════════╪══════════════╪═════════════╡
│ virginica ┆ 6.0          ┆ 2.5         │
│ virginica ┆ 6.1          ┆ 2.5         │
│ virginica ┆ 5.7          ┆ 2.5         │
│ virginica ┆ 5.1          ┆ 2.4         │
│ virginica ┆ 5.6          ┆ 2.4         │
└───────────┴──────────────┴─────────────┘
CPU times: total: 0 ns
Wall time: 2.16 ms
